In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(123)
torch.manual_seed(123)

### Step 1: Generate Data

In [13]:
from synthetic.generate import SingleTaskTreeDepsGenerator

K = 2
M = 10
N = 10000

# Generate the true class balance to be recovered
class_balance = np.random.random(K)
class_balance /= class_balance.sum()

data = SingleTaskTreeDepsGenerator(N, M, K, class_balance, edge_prob=0.15)
print (f"LF Dependencies: {data.E}")
print (f"Class Balance: {data.p}")

LF Dependencies: [(0, 2), (2, 8)]
Class Balance: [0.00599944 0.99400056]


In [14]:
from metal.analysis import lf_summary
lf_summary(data.L,data.Y)

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,"[1, 2]",0.9276,0.9276,0.8468,8523,753,0.918823
1,"[1, 2]",0.8134,0.8134,0.7452,6296,1838,0.774035
2,"[1, 2]",0.9192,0.9192,0.8402,8376,816,0.911227
3,"[1, 2]",0.6813,0.6813,0.6348,3645,3168,0.535007
4,"[1, 2]",0.9363,0.9363,0.8534,8751,612,0.934636
5,"[1, 2]",0.7322,0.7322,0.6750,4585,2737,0.626195
6,"[1, 2]",0.7962,0.7962,0.7323,5775,2187,0.725320
7,"[1, 2]",0.7459,0.7459,0.6886,4815,2644,0.645529
8,"[1, 2]",0.6773,0.6773,0.6312,3468,3305,0.512033
9,"[1, 2]",0.6891,0.6891,0.6393,3859,3032,0.560006


### Step 2: Learn Dependencies using `DependencyLearnerModel`
**TEMP: hardcoding for now**

In [15]:
edges = data.E
L = np.array(data.L.todense())

### Step 3: Recover the class balance using subset of independent LFs

In [16]:
# Generate indices for independent LFs
ind_lfs = []
for i in range(M):
    if i not in list(sum(edges, ())):
        ind_lfs.append(i)
L_ind = L[:,ind_lfs]

In [17]:
%%time
from metal.label_model.class_balance import ClassBalanceModel

model = ClassBalanceModel(K, abstains=True)
model.train_model(L=L_ind, verbose=False)

print(f"Estimated class balance: {model.class_balance}")
print(f"True class balance: {class_balance}")
print()

Estimated class balance: [0.19980003 0.8007491 ]
True class balance: [0.00599944 0.99400056]

CPU times: user 13.9 s, sys: 156 ms, total: 14 s
Wall time: 1.34 s


### Step 4: Train `LabelModel` and predict training labels
**TEMP: not including constraints from Class Balance Model**

In [30]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, class_balance=model.class_balance.astype(float), deps=edges, lr=1e-4, n_epochs=1000, log_train_every=50)

Computing O^{-1}...
Estimating Z...
[50 epo]: TRAIN:[loss=515.940]
[100 epo]: TRAIN:[loss=118.419]
[150 epo]: TRAIN:[loss=82.415]
[200 epo]: TRAIN:[loss=69.957]
[250 epo]: TRAIN:[loss=64.685]
[300 epo]: TRAIN:[loss=62.123]
[350 epo]: TRAIN:[loss=60.756]
[400 epo]: TRAIN:[loss=59.975]
[450 epo]: TRAIN:[loss=59.502]
[500 epo]: TRAIN:[loss=59.200]
[550 epo]: TRAIN:[loss=58.996]
[600 epo]: TRAIN:[loss=58.851]
[650 epo]: TRAIN:[loss=58.742]
[700 epo]: TRAIN:[loss=58.655]
[750 epo]: TRAIN:[loss=58.583]
[800 epo]: TRAIN:[loss=58.520]
[850 epo]: TRAIN:[loss=58.464]
[900 epo]: TRAIN:[loss=58.412]
[950 epo]: TRAIN:[loss=58.362]
[1000 epo]: TRAIN:[loss=58.315]
Finished Training
Estimating \mu...
[50 epo]: TRAIN:[loss=9.529]
[100 epo]: TRAIN:[loss=1.442]
[150 epo]: TRAIN:[loss=0.359]
[200 epo]: TRAIN:[loss=0.271]
[250 epo]: TRAIN:[loss=0.245]
[300 epo]: TRAIN:[loss=0.228]
[350 epo]: TRAIN:[loss=0.214]
[400 epo]: TRAIN:[loss=0.200]
[450 epo]: TRAIN:[loss=0.188]
[500 epo]: TRAIN:[loss=0.178]
[550 ep

In [31]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.995
Precision: 0.750
Recall: 0.207
F1: 0.324
        y=1    y=2   
 l=1    12      4    
 l=2    46    9938   


### Step 4b: Compare to MajorityVote and No Class Balance+Dependencies LabelModel

**Baseline: No Class Balance or Dependencies**

In [20]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.052]
[100 epo]: TRAIN:[loss=0.013]
[150 epo]: TRAIN:[loss=0.007]
[200 epo]: TRAIN:[loss=0.004]
[250 epo]: TRAIN:[loss=0.003]
[300 epo]: TRAIN:[loss=0.002]
[350 epo]: TRAIN:[loss=0.002]
[400 epo]: TRAIN:[loss=0.002]
[450 epo]: TRAIN:[loss=0.002]
[500 epo]: TRAIN:[loss=0.002]
Finished Training
CPU times: user 12 s, sys: 240 ms, total: 12.3 s
Wall time: 443 ms


In [21]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.877
Precision: 0.042
Recall: 0.914
F1: 0.079
        y=1    y=2   
 l=1    53    1224   
 l=2     5    8718   


**Baseline: Majority Vote**

In [22]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(seed=123)
print('Majority Label Voter Metrics:')
scores = mv.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.927
Precision: 0.073
Recall: 0.983
F1: 0.135
        y=1    y=2   
 l=1    57     728   
 l=2     1    9214   
